In [110]:
from numpy import zeros, array
from functools import lru_cache
import pandas as pd
import pymorphy2
import time
import re


morph = pymorphy2.MorphAnalyzer()

In [2]:
lemma_dict = pd.read_csv('C:\\Python programs\\ru_dict_lemma.csv')
print(lemma_dict)

         lemma
0            а
1           аа
2          а-а
3          ааа
4        а-а-а
...        ...
51728  ящерица
51729   ящерка
51730     ящик
51731   ящичек
51732     ящур

[51733 rows x 1 columns]


In [3]:
df_train = pd.read_csv('C:\\Python programs\\dataset_ru_word_lemma2.csv')
print(df_train)

               form          lemma  id_sen
0                 «              «     0.0
1              Если           если     0.0
2          передача       передача     0.0
3          цифровых       цифровой     0.0
4        технологий     технология     0.0
...             ...            ...     ...
19350  провозглашал  провозглашать   999.0
19351          себя           себя   999.0
19352        другом           друг   999.0
19353          мира            мир   999.0
19354             .              .   999.0

[19355 rows x 3 columns]


In [106]:
df_stopwords = pd.read_csv('C:\\Python programs\\stopwords_ru.csv')
print(df_stopwords)

            stopword
0                  а
1          абсолютно
2     авторизоваться
3           активный
4               алло
...              ...
1367               „
1368               “
1369               …
1370               /
1371             ...

[1372 rows x 1 columns]


$$\large\textbf{Расстояние Левенштейна}$$

$$
D(i, j) = 
\begin{cases} 
\max(i, j), \quad \quad \quad \quad \quad \quad  \text{if} \; \min(i, j) = 0  \\
\min( \quad \quad \quad  \quad  \quad \quad \quad \quad \text{otherwise}\\
\quad D(i, j - 1) + 1, \\
\quad D(i - 1, j) + 1, \\
\quad D(i - 1, j - 1) + (W_1[i] \: \neq \: W_2 [j]) \\
)
\end{cases}
$$


In [4]:
@lru_cache
def lev_dist(w1, w2):
    n, m = len(w1), len(w2)
    mat = zeros([n + 1, m + 1], int)
    mat[0, :] = array([i for i in range(m + 1)])
    mat[:, 0] = array([i for i in range(n + 1)])

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            mat[i][j] = min(mat[i - 1][j] + 1, mat[i][j - 1] + 1,
                            mat[i - 1][j - 1] + (w1[i - 1] != w2[j - 1]))
    return mat[n][m]

In [5]:
def lemma_lev(w):
    lemma = ""
    len_w = len(w)
    len_lemma = len_w
    for word in lemma_dict["lemma"]:
        if abs(len_w - len(word)) >= len_lemma:
            continue

        dist = lev_dist(word, w)
        if dist < len_lemma:
            if dist == 0:
                return word
            lemma = word
            len_lemma = dist

    return lemma if lemma != "" else w

In [6]:
print(lemma_lev("гнездился"))

гнездиться


$$ \large\textbf{Поиск по словарю с удалением суффикса}$$

In [7]:
@lru_cache
def lemma_dbsra(w):
    lemma = ""
    for i in range(len(w)):
        for j in range(len(w), i, -1):
            if lemma_dict["lemma"].isin([w[i:j]]).any() and j - i > len(lemma):
                lemma = w[i:j]
    return lemma if lemma != "" else w

In [8]:
print(lemma_dbsra("+"))

+


$$ \large\textbf{Префиксное дерево}$$

In [18]:
class Node(object):
    def __init__(self, value, child=None, key=None, end=False):
        if child is not None and key is not None:
            self.children = {key: child}
        else:
            self.children = {}
        self.value = value
        self._isEnd = end

    def __str__(self):
        return f"keys = {list(self.children.keys())}, value = {self.value}, \
end = {self._isEnd}"

    def addChild(self, value, key):
        self.children[key] = Node(value)

    def hasChild(self, key):
        if key in self.children.keys():
            return True
        return False

    def getChild(self, key):
        return self.children[key]

    def setValue(self, value):
        self.value = value

    def getValue(self):
        return self.value

    def isEnd(self):
        return self._isEnd

    def getKeys(self):
        return list(self.children.keys())

    def setEnd(self):
        self._isEnd = True


class PrefixTrie(object):
    def __init__(self, db=None):
        self.root = Node('')

        if db is not None:
            for word in db:
                self.insert(word, word)

    def insertDict(self, db_form, db_lemma):
        for i in range(len(db_form)):
            self.insert(db_form[i], db_lemma[i])

    def insert(self, key, value):
        node = self.root
        lenght_key = len(key)
        way = ""

        for i in range(lenght_key):
            char = key[i]
            way += char
            if not node.hasChild(char):
                node.addChild(way, char)
            node = node.getChild(char)
        if (not node.isEnd()):
            node.setValue(value)
            node.setEnd()

    def largestPrefix(self, key):
        node = self.root
        lemma = ""
        lenght_key = len(key)

        for i in range(lenght_key):
            char = key[i]
            if not node.hasChild(char):
                break
            node = node.getChild(char)
            if node.isEnd():
                lemma = node.getValue()

        dif = lenght_key - len(lemma)
        if dif != 0:
            depth = 1
            stack = [node.getChild(i) for i in node.getKeys()]
            next_stack = []

            while depth <= max(dif, int(lenght_key / 2)):
                for elem in stack:
                    if elem.isEnd():
                        return elem.getValue()
                    next_stack += [elem.getChild(i) for i in elem.getKeys()]

                stack = next_stack.copy()
                next_stack = []
                depth += 1

        return lemma if lemma != "" else key

    @lru_cache
    def getLemma(self, word):
        prefix = ""
        lemma = word
        len_word = len(word)
        min_dist = len_word

        for i in range(len_word):
            prefix += word[i]
            new_lemma = self.largestPrefix(prefix)
            dist = lev_dist(new_lemma, word)

            if dist < min_dist and dist < len_word // 2:
                if dist == 0:
                    return new_lemma
                lemma = new_lemma
                min_dist = dist

        return lemma

In [19]:
trie = PrefixTrie(lemma_dict["lemma"].apply(str.lower))

In [21]:
print(trie.getLemma("#марка"))

#марка


In [24]:
cnt_PT = 0
cnt_PM2 = 0
len_df = len(df_train.index)

start_time = time.time()
for word_id in range(len_df):
    word = df_train['form'][word_id].lower()
    lemma = df_train['lemma'][word_id].lower()

    # if not lemma_dict["lemma"].isin([lemma]).any():
    #    len_df -= 1
    # else:
    res = trie.getLemma(word)
    if res == lemma:
        cnt_PT += 1

time_PT = round(time.time() - start_time, 3)

start_time = time.time()
for word_id in range(len_df):
    word = df_train['form'][word_id].lower()
    lemma = df_train['lemma'][word_id].lower()
    res = morph.parse(word)[0].normal_form

    if res == lemma:
        cnt_PM2 += 1

time_PM2 = round(time.time() - start_time, 3)

# PrefixTrie : 0.804, DBSRA : 0.583
# PrefixTrie (если вычесть леммы не в словаре) : 0.833
# Pymorphy2 : 0.943
# Pymorphy2 (если вычесть леммы не в словаре) : 0.99
print(f"PrefixTrie : {round(cnt_PT / len_df, 3)}, time : {time_PT}")
print(f"Pymorphy2 : {round(cnt_PM2 / len_df, 3)}, time : {time_PM2}")

PrefixTrie : 0.804, time : 6.067
Pymorphy2 : 0.943, time : 6.083


$$\large \textbf{Расстояние между документами}$$

$$\textbf{Векторизация документов}$$

В stopwordsiso есть 559 русских стоп-слов, найден больший словарь стоп-слов на 1352 слова

In [127]:
def vectorize(list_of_docs):
    """
    list_of_doc = [doc1, doc2, ...]
    doc1 = [word1, word2, ...]
    """

    count_of_docs = len(list_of_docs)
    res_df = pd.DataFrame()
    zero_df = pd.DataFrame({'None': [0 for i in range(count_of_docs)]})
    num_reg_exp = r'[-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?'

    for id_doc in range(count_of_docs):
        for word in list_of_docs[id_doc]:
            lemma = morph.parse(word)[0].normal_form

            if df_stopwords['stopword'].isin([lemma, word]).any() or \
                    re.fullmatch(num_reg_exp, word) is not None:
                continue

            if lemma not in res_df.columns:
                res_df = pd.concat([res_df,
                                   zero_df.rename(columns={'None': lemma})],
                                   axis=1)

            res_df[lemma][id_doc] += 1

    return res_df

In [128]:
print(vectorize([['я', 'сидел', 'и', 'глядел'], ['ты', 'сидел', 'и', 'глядел']]))

   глядеть
0        1
1        1


In [87]:
df_soze = df_train.shape[0]
list_docs = [list(df_train.loc[df_train['id_sen'] == id_sen, 'form']) for id_sen in range(1000)]

In [116]:
vect_list = vectorize(list_docs)

In [117]:
print(vect_list)

     передача  цифровой  сша  впервые  мирный  власть  написать  корь  \
0           2         1    1        1       1       1         1     1   
1           1         0    0        0       0       0         0     0   
2           0         0    0        0       0       0         0     0   
3           0         0    0        0       0       0         0     0   
4           0         0    0        0       0       0         0     0   
..        ...       ...  ...      ...     ...     ...       ...   ...   
995         0         0    0        0       0       0         0     0   
996         0         0    0        0       0       0         0     0   
997         0         0    0        0       0       0         0     0   
998         0         0    0        0       0       0         0     0   
999         0         0    0        0       0       0         0     0   

     шульман  специальный  ...  окончательно  баллотироваться  консул  отвод  \
0          1            1  ...             

In [149]:
@lru_cache
def norm(vect):
    return sum([elem ** 2 for elem in vect]) ** 0.5


def prod_vect(vect1, vect2):
    return sum([vect1[i] * vect2[i] for i in range(len(vect1))])


def cosine_docs(vect1, vect2):
    return prod_vect(vect1, vect2) / (norm(vect1) * norm(vect2))

In [151]:
for i in range(1000):
    vect1 = tuple(vect_list.values[i])
    for j in range(i + 1, 1000):
        vect2 = tuple(vect_list.values[j])
        cos_value = cosine_docs(vect1, vect2)

        if cos_value > 0.5:
            print(list_docs[i])
            print(list_docs[j])
            print(cos_value)
            print()

C:\Users\user\AppData\Local\Temp\ipykernel_7016\2316977802.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return prod_vect(vect1, vect2) / (norm(vect1) * norm(vect2))


['Максимально', 'допустимая', 'сумма', 'на', 'одного', 'человека', '—', '5000', 'долларов', 'США', '.']
['Сумма', 'сдельной', 'заработной', 'платы', 'и', 'бонусов', 'в', 'том', 'году', 'составила', '1,5', 'миллиона', 'долларов', 'США', ',', 'примерно', 'на', 'том', 'же', 'уровне', ',', 'что', 'и', 'в', '2015', '—', '2016', 'гг', '.']
0.5163977794943222

['Нужно', 'провести', 'параллель', 'между', 'играми', 'и', 'нашей', 'повседневной', 'жизнью', '.']
['Как', 'полагают', 'историки', ',', 'последние', 'Олимпийские', 'игры', 'были', 'проведены', 'в', '393', 'году', 'н.', 'э', '.']
0.5163977794943222

['Руководители', 'также', 'получили', 'так', 'называемую', '«', 'сдельную', 'заработную', 'плату', '»', 'за', 'выполнение', 'или', 'превышение', 'ожиданий', ',', 'разделив', 'между', 'собой', 'сумму', 'в', '1,5', 'миллиона', 'долларов', 'США', ',', 'или', 'в', 'среднем', '15000', 'долларов', 'США', 'каждый', '.']
['Сумма', 'сдельной', 'заработной', 'платы', 'и', 'бонусов', 'в', 'том', 'году',